In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset["Purchased"].value_counts()
#refer how many class classified
#to find balanced or imbalanced
#unique entry &counts---->(0-143,1-143)->balanced
#(0-257,1-143)->imbalanced

0    257
1    143
Name: Purchased, dtype: int64

In [7]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [8]:
indep.shape#400-length ,3-i/p column

(400, 3)

In [9]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [10]:
#split into training & test set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)



In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [12]:
#replace regression to classifier for classification
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
param_grid={'criterion':['gini', 'entropy', 'log_loss'],'splitter':['best', 'random']}
grid=GridSearchCV(DecisionTreeClassifier (probability=True),param_grid,refit=True,verbose=3,n_jobs=1,scoring='f1_weighted')
#f1_weighted suitable for all balanced &imbalance including precision,recall)
#fitting model for grid search
grid.fit(x_train,y_train)


TypeError: __init__() got an unexpected keyword argument 'probability'

In [ ]:
#print best parameter after tuning
#print(grid.best_params_)
re=grid.cv_results_
grid_predictions=grid.predict(x_test)



In [ ]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)#optional

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [ ]:
# 79,0 correct diagonal,41,0 wrong diagonal
#type1 error should low
print(cm)

In [ ]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [ ]:
#overall performance(accuracy)=0.89
print(clf_report)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])#receiver operating chareteristics area under curve(roc_auc)

In [ ]:
table = pd.DataFrame.from_dict(re)

In [ ]:
table

In [ ]:
age_input=int(input("Age:"))
estimated_salary=int(input("salary:"))
gender_male_input=int(input("sex male 0 or 1:"))


In [ ]:
Future_Prediction=grid.predict([[age_input,estimated_salary,gender_male_input]])
print("Future_Prediction{}".format(Future_Prediction))


In [ ]:
import pickle
filename="finalized_model_Grid.sav"
pickle.dump(grid,open(filename,'wb'))

In [ ]:
loaded_model=pickle.load(open("finalized_model_Grid.sav",'rb'))
result=loaded_model.predict([[36,33000,0]])

In [ ]:
result